#### If using AMD GPU, switch backend to PlaidML library:

In [ ]:
import os
os.environ['KERAS_BACKEND']='plaidml.keras.backend'

In [24]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import time
import pickle
from keras.models import load_model

#### Define Experiments

In [15]:
conv_layers = [6]      # number of conv layers
layer_sizes = [32]     # number of nodes in a layer
dense_layers = [2]     # number of dense layers

#### Load input data

In [38]:
pickle_in = open('../Dataset/df_all.pickle', 'rb')
df_train, df_test = pickle.load(pickle_in)

In [39]:
# The Keras ImageDataGenerator uses string type data label
df_train['gender'] = df_train.gender.astype(str)
df_test['gender'] = df_test.gender.astype(str)

In [40]:
print(df_train.shape, df_test.shape)

(429648, 10) (22613, 10)


In [41]:
df_train.dtypes

path                object
id                  uint16
name                object
dob         datetime64[ns]
gender              object
score1             float64
score2             float64
pic_date    datetime64[ns]
region              object
age                float64
dtype: object

We will be using a generator to feed model with images, the X would be the path to these images. y will be the gender label.

In [42]:
image_reshape_size = 120
input_image_root_dir = '../Dataset/imdb_crop/' # Don't forget the ending slash

In [43]:
from keras import backend as K
K.set_image_data_format('channels_last')
batch_size = 32
inputShape = (image_reshape_size, image_reshape_size, 1)

In [44]:
df_train.head(3)

,path,id,name,dob,gender,score1,score2,pic_date,region,age
82910,04/nm0000704_rm2567219200_1981-1-28_2011.jpg,5674,Elijah Wood,1981-01-28,1,2.772975,1.210166,2011-01-01,"[425.43, 98.31, 487.20000000000005, 160.08]",29.925324
258557,80/nm1429380_rm1298313472_1990-4-18_2011.jpg,2599,Britt Robertson,1990-04-18,0,-inf,NaN,2011-01-01,"[1, 1, 1328, 2000]",20.706791
265450,23/nm1724323_rm2543041024_1979-7-16_2009.jpg,8609,Jayma Mays,1979-07-16,0,3.557743,NaN,2009-01-01,"[224.7283354218222, 48.83964330467618, 295.812...",29.465355


#### Set up input image generator using flow_from_dataframe

In [45]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_dataframe(dataframe=df_train,
                                            directory=input_image_root_dir,
                                            x_col="path", y_col="gender",
                                            subset="training",
                                            class_mode="binary",
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=batch_size,
                                            seed=1,
                                            shuffle=True)

val_generator = datagen.flow_from_dataframe(dataframe=df_train,
                                            directory=input_image_root_dir,
                                            x_col="path", y_col="gender",
                                            subset="validation",
                                            class_mode="binary",
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=batch_size,
                                            seed=1,
                                            shuffle=True)

test_generator = datagen.flow_from_dataframe(dataframe=df_test, 
                                            directory=input_image_root_dir, 
                                            x_col="path", y_col=None, 
                                            class_mode=None, 
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=1,
                                            shuffle=False)

Found 386684 images belonging to 2 classes.
Found 42964 images belonging to 2 classes.
Found 22613 images.


#### (1) Run training experiments

In [ ]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            
            NAME = 'BN-{}-conv-{}-node-{}-dens-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))  # model name with timestamp
            print(NAME) 
            
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            checkpoint = ModelCheckpoint('weights/{}'.format(NAME), monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
            callbacks = [tensorboard, checkpoint]
            
            model = Sequential()
            
            # first layer
            model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu", input_shape=inputShape))
            model.add(BatchNormalization())
            model.add(MaxPooling2D(pool_size=(3,3)))
            
            # sets up additional # of conv layers
            for _ in range(conv_layer - 1):
                layer_size *= 2
                model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu"))
                model.add(BatchNormalization())
                model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu"))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(pool_size=(2,2)))
                model.add(Dropout(0.25))
            
            model.add(Flatten())
            
            layer_size *= 4 # to get the dense layer to be 8X of last output size
            
            # sets up # of dense layers
            for _ in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.5))
            
            # output layer
            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            opt = Adam(lr=0.001)
            model.compile(loss='binary_crossentropy', 
                          optimizer=opt,
                          metrics=['accuracy'])

            model.fit_generator(generator=train_generator,
                                steps_per_epoch=(train_generator.n // train_generator.batch_size),
                                callbacks = callbacks,
                                validation_data=val_generator,
                                validation_steps=(val_generator.n // val_generator.batch_size),
                                epochs=50,
                                use_multiprocessing=False,
                                workers=4)
            
            filepath = NAME + '.h5'
            model.save(filepath)

BN-5-conv-32-node-2-dens-1554148809
Epoch 1/50
12083/12083 [==============================] - 15893s 1s/step - loss: 0.6381 - acc: 0.6845 - val_loss: 0.6569 - val_acc: 0.6881
Epoch 2/50
12083/12083 [==============================] - 15759s 1s/step - loss: 0.5761 - acc: 0.7328 - val_loss: 0.5420 - val_acc: 0.7472
Epoch 3/50
12083/12083 [==============================] - 15993s 1s/step - loss: 0.5625 - acc: 0.7350 - val_loss: 0.6274 - val_acc: 0.7454
Epoch 4/50
12083/12083 [==============================] - 16054s 1s/step - loss: 0.5413 - acc: 0.7485 - val_loss: 0.5725 - val_acc: 0.7485
Epoch 5/50
12083/12083 [==============================] - 16288s 1s/step - loss: 0.5251 - acc: 0.7573 - val_loss: 0.5472 - val_acc: 0.7385
Epoch 6/50
12083/12083 [==============================] - 16458s 1s/step - loss: 0.5156 - acc: 0.7621 - val_loss: 0.5314 - val_acc: 0.7630
Epoch 7/50
12083/12083 [==============================] - 16846s 1s/step - loss: 0.5092 - acc: 0.7657 - val_loss: 0.5296 - val_acc

#### Load model, resume training

In [11]:
inputFile = 'BN-5-conv-32-node-2-dens-1553895953-3.h5'
model = load_model(inputFile)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [12]:
saveAs = 'BN-5-conv-32-node-2-dens-1553895953-4Lab'

In [ ]:
from keras.callbacks import ModelCheckpoint

NAME = saveAs
print(NAME) 

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
callbacks = [tensorboard]

opt = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])

model.fit_generator(generator=train_generator,
                    steps_per_epoch=(train_generator.n // train_generator.batch_size),
                    callbacks = callbacks,
                    validation_data=val_generator,
                    validation_steps=(val_generator.n // val_generator.batch_size),
                    epochs=5,
                    use_multiprocessing=False,
                    workers=8)

filepath = NAME + '.h5'
model.save(filepath)

#### Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix
test_generator.reset()
pred=model.predict_generator(test_generator,
                            steps=test_generator.n//test_generator.batch_size,
                            verbose=1)


In [ ]:
y_true = df_test.gender.astype(int)
y_pred = [1 if x>=0.5 else 0 for x in pred]
cm = confusion_matrix(y_true, y_pred)

In [ ]:
cm

In [ ]:
TN = cm[0][0]
TP = cm[1][1]
FN = cm[1][0]
FP = cm[0][1]

Specificity

In [ ]:
TN/(TN+FP)

Precision

In [ ]:
TP/(TP+FP)

Recall

In [ ]:
TP/FN

Accuracy

In [ ]:
(TP+TN)/(TN+TP+FN+FP)

#### Live Demo

In [ ]:
model = load_model('BN-5-conv-32-node-2-dens-1553895953.h5')

In [ ]:
def prepare(filepath):
    IMG_SIZE = image_reshape_size
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
prediction = model.predict(prepare('picture.jpg'))
print(prediction)